In [1]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle

In [2]:
# Test
# Test 2
# Test 3

URL = 'https://fbref.com/en/comps/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
# Club International Cups
club_int_comps = []

for j in soup.find_all('tbody')[0].find_all("tr", {"class": "gender-m"}):
    if (j.find('td', {'data-stat':'tier'}) != None):
        if (j.find('td', {'data-stat':'tier'}).text == '1st') | (j.find('td', {'data-stat':'tier'}).text == '2nd'):
            club_int_comps.append(j.find('a')['href'])

club_int_comps

['/en/comps/14/history/Copa-Libertadores-Seasons',
 '/en/comps/205/history/Copa-Sudamericana-Seasons',
 '/en/comps/8/history/UEFA-Champions-League-Seasons',
 '/en/comps/19/history/UEFA-Europa-League-Seasons']

In [4]:
# National Team Competitions
national_comps = []

for j in soup.find_all('tbody')[1].find_all("tr", {"class": "gender-m"}):
    if (j.find('td', {'data-stat':'tier'}) != None):
        if (j.find('td', {'data-stat':'tier'}).text == '1st') | (j.find('td', {'data-stat':'tier'}).text == '2nd'):
            national_comps.append(j.find('a')['href'])

national_comps

['/en/comps/1/history/World-Cup-Seasons',
 '/en/comps/656/history/Africa-Cup-of-Nations-Seasons',
 '/en/comps/681/history/CONCACAF-Gold-Cup-Seasons',
 '/en/comps/257/history/OFC-Nations-Cup-Seasons',
 '/en/comps/676/history/European-Championship-Seasons',
 '/en/comps/664/history/AFC-Asian-Cup-Seasons']

In [5]:
# Domestic Tier 1 Comps
dom_tier1_comps = []

for j in soup.find_all('tbody')[3].find_all("tr", {"class": "gender-m"}):
    if (j.find('td') != None):
        dom_tier1_comps.append(j.find('a')['href'])

dom_tier1_comps

['/en/comps/21/history/Primera-Division-Seasons',
 '/en/comps/65/history/A-League-Seasons',
 '/en/comps/56/history/Austrian-Bundesliga-Seasons',
 '/en/comps/37/history/Belgian-First-Division-A-Seasons',
 '/en/comps/74/history/Primera-Division-Seasons',
 '/en/comps/24/history/Serie-A-Seasons',
 '/en/comps/67/history/Bulgarian-First-League-Seasons',
 '/en/comps/211/history/Canadian-Premier-League-Seasons',
 '/en/comps/35/history/Primera-Division-Seasons',
 '/en/comps/62/history/Super-League-Seasons',
 '/en/comps/41/history/Primera-A-Seasons',
 '/en/comps/63/history/1-HNL-Seasons',
 '/en/comps/66/history/Czech-First-League-Seasons',
 '/en/comps/50/history/Superliga-Seasons',
 '/en/comps/58/history/Serie-A-Seasons',
 '/en/comps/9/history/Premier-League-Seasons',
 '/en/comps/12/history/La-Liga-Seasons',
 '/en/comps/43/history/Veikkausliiga-Seasons',
 '/en/comps/13/history/Ligue-1-Seasons',
 '/en/comps/20/history/Bundesliga-Seasons',
 '/en/comps/27/history/Super-League-Greece-Seasons',
 '/en

In [6]:
all_comps = club_int_comps + national_comps + dom_tier1_comps

len(all_comps)

58

In [7]:
def get_all_seasons(url):
    URL = 'https://fbref.com/' + url
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    url_list = []
    
    for row in soup.find_all('tr'):
        if row.find('th',{"scope":"row"}) != None:
            url_list.append((row.find('a')['href']))
    
    return url_list

In [8]:
all_seasons = []

for i in all_comps:
    league_seasons = get_all_seasons(i)
    all_seasons += league_seasons

In [9]:
len(all_seasons)

886

In [10]:
# keep just in case

# def get_all_player_urls(url_list):
    
#     final_player_urls = []

#     for i in url_list:
#         URL_test = 'https://fbref.com/'+ i
#         page = requests.get(URL_test)
#         soup = BeautifulSoup(page.content, 'html.parser')
#         td1 = soup.find(lambda t: t.text.strip()=='Standard Stats')
#         td2 = td1.find_next('a')
#         url_to_add = td2['href']
#         final_player_urls.append(url_to_add)

#     return final_player_urls

In [11]:
def get_all_player_urls(url_list):
    
    final_player_urls = []

    for i in url_list:
        final_pos = i.rindex('/')

        final_player_urls.append(i[:final_pos+1]+'stats/'+i[final_pos+1:])

    return final_player_urls

In [12]:
final_stats = get_all_player_urls(all_seasons)

In [13]:
final_stats[0]

'/en/comps/14/stats/Copa-Libertadores-Stats'

In [14]:
# all_standard_urls = get_all_player_urls(all_seasons)

In [15]:
def get_player_table(url):
    url = 'https://fbref.com/' + url
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")
    player_table = all_tables[2]

    return player_table

In [16]:
def get_player_urls(url_list):
    
    player_url_list = []

    for url in url_list:
        url = 'https://fbref.com/' + url
        res = requests.get(url)
        ## The next two lines get around the issue with comments breaking the parsing.
        comm = re.compile("<!--|-->")
        soup = BeautifulSoup(comm.sub("",res.text),'lxml')
        all_tables = soup.findAll("tbody")
        try:
            player_table = all_tables[2]
        except:
            next

        for row in player_table.find_all('a'):
            if "matchlogs" in row['href']:
                player_url_list.append(row['href'])

    return player_url_list

In [17]:
stats = ["player","nationality","position","squad","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]

In [18]:
def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    for row in rows_player:
        if(row.find('th',{"scope":"row"}) != None):
    
            for f in features_wanted_player:
                cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)

    return df_player

In [21]:
# get_frame(stats, testing)

In [20]:
player_table = get_player_urls(final_stats)

len(player_table)

345527

In [22]:
# Saving player_table as pickle file
import pickle

with open('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Pickle Files/player_table.pickle', 'wb') as handle:
    pickle.dump(player_table, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
# Reading player_table pickle file

with open('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Pickle Files/player_table.pickle', 'rb') as handle:
    pickle_player_table = pickle.load(handle)

pickle_player_table

345527

In [97]:
df = pd.DataFrame([])

for player in pickle_player_table:
    url = 'https://fbref.com/' + player
    res = requests.get(url)

    soup = BeautifulSoup(res.text,'lxml')
    
    new_table = pd.read_html(url)[0]
    new_header = new_table.columns.droplevel()
    new_table.columns = new_header
    # new_table.fillna(0)
    new_table['name'] = soup.title.string

    # new_table = new_table.dropna(axis=0, how ='all').reset_index()

    df = df.append(new_table)

df.head(50)

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,...,Fld,Off,Crs,TklW,Int,OG,PKwon,PKcon,Match Report,name
0,2021-01-17,Sun,Primera Div,Apertura regular season,Away,L 0–2,Santa Fe,Nacional,N,NaN,...,1,0,1,0,0,0,NaN,NaN,Match Report,Johan Caballero 2021 Match Logs | FBref.com
1,2021-01-30,Sat,Primera Div,Apertura regular season,Away,W 2–0,Santa Fe,Rionegro,Y,AM,...,2,0,8,0,0,0,NaN,NaN,Match Report,Johan Caballero 2021 Match Logs | FBref.com
2,2021-02-04,Thu,Primera Div,Apertura regular season,Home,W 1–0,Santa Fe,Boyacá Patriot,Y,AM,...,0,1,8,2,1,0,NaN,NaN,Match Report,Johan Caballero 2021 Match Logs | FBref.com
3,2021-02-07,Sun,Primera Div,Apertura regular season,Away,W 1–0,Santa Fe,Jaguares,Y,AM,...,0,1,6,0,0,0,NaN,NaN,Match Report,Johan Caballero 2021 Match Logs | FBref.com
4,2021-02-14,Sun,Primera Div,Apertura regular season,Home,W 2–0,Santa Fe,CA Bucaramanga,Y,AM,...,3,1,4,1,0,0,NaN,NaN,Match Report,Johan Caballero 2021 Match Logs | FBref.com
5,2021-02-18,Thu,Primera Div,Apertura regular season,Away,D 0–0,Santa Fe,CD América,Y,AM,...,1,0,5,1,0,0,NaN,NaN,Match Report,Johan Caballero 2021 Match Logs | FBref.com
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Johan Caballero 2021 Match Logs | FBref.com
7,2021-03-04,Thu,Primera Div,Apertura regular season,Home,W 3–2,Santa Fe,Tolima,Y,AM,...,2,0,2,0,0,0,NaN,NaN,Match Report,Johan Caballero 2021 Match Logs | FBref.com
8,2021-03-08,Mon,Primera Div,Apertura regular season,Home,D 2–2,Santa Fe,Pasto,Y,FW,...,1,1,4,0,1,0,NaN,NaN,Match Report,Johan Caballero 2021 Match Logs | FBref.com
9,2021-03-20,Sat,Primera Div,Apertura regular season,Away,L 1–2,Santa Fe,Envigado,N,NaN,...,0,0,4,0,2,0,NaN,NaN,Match Report,Johan Caballero 2021 Match Logs | FBref.com
